# Instance creator utility (realtime)

## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
import sys
import copy
import random

import xml.dom.minidom
import xml.etree.ElementTree as ET

sys.path.append('..')
from models.Fleet import Fleet, from_xml

In [2]:
def TSubElement(parent, tag, attrib={}, text=None, **extra):
    element = ET.SubElement(parent, tag, attrib, **extra)
    if text:
        element.text = text
    return element

## ET handler methods
### Info

In [3]:
def createInfo(root, name='', description=''):
    info = TSubElement(root, 'info')
    _name = TSubElement(info, 'name', text=name)
    _description = TSubElement(info, 'description', text=description)
    return

### Network

In [4]:
def createNetwork(root, depots, customers, charging_stations, 
                  cx_low=-1.0, cx_high=1.0, cy_low=-1.0, cy_high=1.0, 
                  request_min=0.01, request_max=0.8,
                  spent_time_min=2.0, spent_time_max=15.0,
                  tw_low_low=600.0, tw_low_high=960.0, tw_min_width= 20.0, tw_max_width=120.0, 
                  travel_time_min=3., travel_time_max=35., 
                  energy_consumption_min=2.0, energy_consumption_max=10.0,
                  max_capacity_charge_stations=4):
    
    network = TSubElement(root, 'network')
    nodes = TSubElement(network, 'nodes') # stores the nodes
    edges = TSubElement(network, 'edges') # stores edges
    info = TSubElement(network, 'info') # stores info about the network
    technologies = TSubElement(network, 'technologies') # stores info about CS techonologies
    
    # depot nodes
    for i in depots:
        attr = {'id': str(i), 'type': str(0)}
        node = TSubElement(nodes, 'node', attrib=attr)

    # customer nodes
    for i in customers:
        attr = {'id': str(i), 'type': str(1)}
        node = TSubElement(nodes, 'node', attrib=attr)
        
        request = np.random.uniform(request_min, request_max)
        node.set('demand', '{:.2f}'.format(request))
        
        spent_time = np.random.uniform(spent_time_min, spent_time_max)
        node.set('spent_time', '{:.2f}'.format(spent_time))
        
        tw_low = np.random.uniform(tw_low_low, tw_low_high)
        tw_upp = tw_low + np.random.uniform(tw_min_width, tw_max_width)
        node.set('time_window_low', '{:.2f}'.format(tw_low))
        node.set('time_window_upp', '{:.2f}'.format(tw_upp))

    # CS nodes
    for i in charging_stations:
        attr = {'id': str(i), 'type': str(2)}
        node = TSubElement(nodes, 'node', attrib=attr)
        
        technology = np.random.randint(1, 4) # choose among 1, 2 and 3
        node.set('technology', str(technology))
        
        capacity = np.random.randint(1, max_capacity_charge_stations+1)
        node.set('capacity', str(capacity))
    
    # add coordinates (random)
    for node in nodes:
        cx = np.random.uniform(cx_low, cx_high)
        cy = np.random.uniform(cy_low, cy_high)
        node.set('cx', '{:.2f}'.format(cx))
        node.set('cy', '{:.2f}'.format(cy))
        
    # edges
    for i in nodes:
        nodeFrom = TSubElement(edges, 'node_from', attrib={'id': i.get('id')})
        for j in nodes:
            nodeTo = TSubElement(nodeFrom, 'node_to', attrib={'id': j.get('id')})
            
            if i == j:
                travelTime = 0.
                energyConsumption = 0.
            else:
                travelTime = np.random.uniform(travel_time_min, travel_time_max)
                energyConsumption = np.random.uniform(energy_consumption_min, energy_consumption_max)
                
            nodeTo.set('travel_time', '{:.2f}'.format(travelTime))
            nodeTo.set('energy_consumption', '{:.2f}'.format(energyConsumption))
            
    # technologies
    _tech1 = TSubElement(technologies, 'technology', attrib={'type': str(1)})
    _tech2 = TSubElement(technologies, 'technology', attrib={'type': str(2)})
    _tech3 = TSubElement(technologies, 'technology', attrib={'type': str(3)})
    
    tech1 = {'0.0': '0.0', '20.0': '75.0', '50.0': '80.0', '120.0': '100.0'}
    tech2 = {'0.0': '0.0', '15.0': '75.0', '40.0': '80.0', '80.0': '100.0'}
    tech3 = {'0.0': '0.0', '10.0': '75.0', '25.0': '80.0', '40.0': '100.0'}
    
    _tech_list = [_tech1, _tech2, _tech3]
    tech_list = [tech1, tech2, tech3]
    
    for _tech, tech in zip(_tech_list, tech_list):
        for chargingTime, battLevel in tech.items():
            breakPoint = TSubElement(_tech, 'breakpoint')
            breakPoint.set('charging_time', chargingTime)
            breakPoint.set('battery_level', battLevel)
    

### Fleet

In [5]:
def createFleet(root, fleet):
    _fleet = TSubElement(root, 'fleet')
    for ev_id, attrib in fleet.items():
        attrib['id'] = str(ev_id)
        _ev = TSubElement(_fleet, 'electric_vehicle', attrib=attrib)
    return

def fleet_assign_customers(root, info):
    _fleet = root.find('fleet')
    for id_ev, customers in info.items():
        for _ev in _fleet:
            if id_ev == _ev.get('id'):
                _assgn_customers = TSubElement(_ev, 'assigned_customers')
                for customer in customers:
                    _customer = TSubElement(_assgn_customers, 'node', attrib={'id':str(customer)})
                break
                

def fleet_update(root, update_info):
    _fleet = root.find('fleet')
    for ev_id, info in update_info.items():
        _ev = _fleet[ev_id-1]
        _previous_sequence = TSubElement(_ev, 'previous_sequence')
        sk_list = info['previous_sequence'][0]
        lk_list = info['previous_sequence'][1]
        for sk, lk in zip(sk_list, lk_list):
            attrib = {'sk':str(sk), 'lk':str(lk)}
            _event = TSubElement(_previous_sequence, 'event', attrib=attrib)
        
        _critical_point = TSubElement(_ev, 'critical_point', info['critical_point'])
            
    


In [6]:
def createTree(numDepots, numCustomers, numChargeStations, attribsEV):
    dataElement = ET.Element('instance')
    
    createInfo(dataElement)
    createNetwork(dataElement, numDepots, numCustomers, numChargeStations)
    fleetElement = createFleet(dataElement, attribsEV)
    
    return ET.ElementTree(dataElement)

### Folder and file path

In [7]:
#instanceName = 'd'+str(numDepot)+'c'+str(numCustomer)+'cs'+str(numCS)+'_ev'+str(numEV)
instanceName = '10C_2CS_1D_2EV'
folderPath = '../data/GA_implementation_xml/'+instanceName
filePath = folderPath+'/'+instanceName+'.xml'
filePath_realtime = folderPath+'/'+instanceName+'_realtime.xml'
filePath_already_assigned = folderPath+'/'+instanceName+'_already_assigned.xml'

try:
    os.mkdir(folderPath)
except FileExistsError:
    pass
print('Assignation XML path:', filePath)
print('Assigned XML path   :', filePath_already_assigned)
print('Real-time XML path  :', filePath_realtime)


Assignation XML path: ../data/GA_implementation_xml/10C_2CS_1D_2EV/10C_2CS_1D_2EV.xml
Assigned XML path   : ../data/GA_implementation_xml/10C_2CS_1D_2EV/10C_2CS_1D_2EV_already_assigned.xml
Real-time XML path  : ../data/GA_implementation_xml/10C_2CS_1D_2EV/10C_2CS_1D_2EV_realtime.xml


### Create a new instance or open an existing one
Choose if creating a new one by setting `new_instance` to `True`. In contrary, set it to `False` to open an existing one

In [8]:
new_instance = False
print_pretty = True

# Parameter of new instance
numDepot = 1
numCustomer = 20
numCS = 4
numEV = 4

if new_instance:
    # Nodes
    depots = range(numDepot)
    customers = range(numDepot, numDepot+numCustomer)
    charging_stations = range(numDepot+numCustomer, numDepot+numCustomer+numCS)

    # Fleet
    attrib = {'max_payload': '2.5', 
              'max_tour_duration': '250.0', 
              'battery_capacity': '220.0', 
              'alpha_down': '35.0', 'alpha_up':'85.0'}

    evs ={x:attrib for x in range(numEV)}

    # Create tree
    dataTree = createTree(depots, customers, charging_stations, evs)

    # Save tree
    print('Saving to:', folderPath)
    dataTree.write(filePath)
    
    # A random customer assignation    
    customer_assignation = {}

    ids_customer = copy.deepcopy(list(customers))
    customers_per_car = [int(len(ids_customer) / numEV)] * numEV

    if len(ids_customer) % numEV != 0:
        customers_per_car[-1] = int(len(ids_customer) / numEV) + 1

    for i, j in enumerate(customers_per_car):
        print('Car', i, 'must visit', j, 'customer/s')
    print('\n')

    for id_car, num_customers in enumerate(customers_per_car):
        ids_customer_to_visit = []
        for j in range(0, num_customers):
            index = random.randint(0, len(ids_customer) - 1)
            ids_customer_to_visit.append(ids_customer.pop(index))
        print('Car', id_car, 'must visit customers_per_vehicle with ID:', ids_customer_to_visit)
        customer_assignation[str(id_car)] = ids_customer_to_visit

    dataTree_assigned = dataTree
    fleet_assign_customers(dataTree_assigned, customer_assignation)
    dataTree_realtime = dataTree_assigned

    print('Saving assignation to:', filePath_already_assigned)
    dataTree_assigned.write(filePath_already_assigned)
    
    print('Saving realtime to:', filePath_realtime)
    dataTree_realtime.write(filePath_realtime)

    if print_pretty:
        xml_pretty = xml.dom.minidom.parse(filePath_realtime).toprettyxml()
        with open(filePath_realtime, 'w') as file:
            file.write(xml_pretty)
        with open(filePath_already_assigned, 'w') as file:
            file.write(xml_pretty)
            
else:
    dataTree = ET.parse(filePath)
    dataTree_assigned = ET.parse(filePath_already_assigned)
    dataTree_realtime = ET.parse(filePath_realtime)

Saving to: ../data/GA_implementation_xml/10C_2CS_1D_2EV
Car 0 must visit 5 customer/s
Car 1 must visit 5 customer/s
Car 2 must visit 5 customer/s
Car 3 must visit 5 customer/s


Car 0 must visit customers_per_vehicle with ID: [14, 15, 3, 6, 13]
Car 1 must visit customers_per_vehicle with ID: [18, 11, 10, 19, 1]
Car 2 must visit customers_per_vehicle with ID: [8, 2, 7, 16, 9]
Car 3 must visit customers_per_vehicle with ID: [5, 4, 12, 17, 20]
Saving assignation to: ../data/GA_implementation_xml/10C_2CS_1D_2EV/10C_2CS_1D_2EV_already_assigned.xml
Saving realtime to: ../data/GA_implementation_xml/10C_2CS_1D_2EV/10C_2CS_1D_2EV_realtime.xml


## Modify real-time file
Observation is made by getting position of vehicles. A vehicle can be travelling from a node to another node or can be in a stop. 

If the vehicle is in a stop, observation must report this as case 0 and include: the portion of time (known as $\eta$) it has been at the node according to the time the vehicle reached and the time it expects to leave, and the node the vehicle is at.

If the vehicle is travelling, observation must report this as case 1 and include: the portion of the edge that has been trevelled (known as $\eta$), the node where the vehicle departs and the node where the vehicle reaches.

Critical points are calculated as follows:
1. Observe the network
2. For each vehicle to route, find the time it is expected to reach the node next to the observation
3. If the time expected to reach the next node is higher than the time it takes to run GA, assign it as critical point; else, iterate next nodes until the previous condition is satisfied.
4. Save all critical points according to EV ids

If a critical point is at most consider_until nodes to reach the end of operation, it means that the EV is about to end the operation. Thus, it is not worth recalculate a new route for that EV. The critical point shoul return with k=-1, which means that this EV must not be considered for routing.

In [8]:
# This function calculates and displays each eta from given iterated fleet
def observe(fleet, total_time):
    #print(f'Total time: {total_time}')
    observation = {}
    for ev_id, ev in fleet.vehicles.items():
        reaching_times = ev.state_reaching[0, :] - ev.state_reaching[0, 0]
        leaving_times = ev.state_leaving[0, :] - ev.state_reaching[0, 0]
        for k, (reach_time, leave_time) in enumerate(zip(reaching_times, leaving_times)):
            node = ev.route[0][k]
            p = ev.state_reaching[1, k]
            q = ev.route[1][k]
            tspent = fleet.network.spent_time(node, p, q)
            if reach_time <= total_time <= leave_time:
                # case in node
                case = 0
                eta = (total_time - reach_time)/(leave_time - reach_time)
                #print(f'CASE NODE | node: {node}  eta: {eta}  expected leaving time: {tspent*(1-eta)}')
                values = (case, eta, node)
                break
            elif total_time < reach_time:
                # case in edge
                case = 1
                eta  = (total_time - leaving_times[k-1])/(reach_time - leaving_times[k-1])
                #print(f'CASE EDGE - Vehicle {ev_id} leaves node {ev.route[0][k-1]} at {leaving_times[k-1]} and reaches node {ev.route[0][k]} at {reach_time} (eta: {eta} )')
                values = (case, eta, ev.route[0][k-1], ev.route[0][k])
                break
        observation[ev_id] = values
    return observation
                
def critical_points(fleet, observation, safe_time, consider_until=3):
    network = fleet.network
    cp = {}
    for id_ev, values in observation.items():
        ev = fleet.vehicles[id_ev]
        node_seq, chrg_seq = ev.route
        reaching_times = ev.state_reaching[0, :]
        leaving_times = ev.state_leaving[0, :]
        #print(f'  STATE REACHING\n{ev.state_reaching}')
        #print(f'  STATE LEAVING\n{ev.state_leaving}')
        if values[0] == 0:
            # case in node
            print('Case NODE')
            eta, node = values[1], values[2]
            k_init = node_seq.index(node)
            p, q = ev.state_reaching[1, k_init], chrg_seq[k_init]
            tspent = network.spent_time(node, p, q)
            
            reach_seq = reaching_times[k_init:] - reaching_times[k_init]
            leav_seq = leaving_times[k_init:] - reaching_times[k_init]
            
            offset = tspent*eta
            reach_seq[0] = offset
            
            reach_seq -= offset
            leav_seq -= offset
            
            #print('reach seq', reach_seq)
            #print('leave seq', leav_seq)
            #print('eta', eta)
            #print('node', node)
            #print('spent time', tspent)
            #print('p', p, '  q', q )
            
        else:
            # case in edge
            #print('Case EDGE')
            eta, node_from, node_to = values[1], values[2], values[3]
            k_init = node_seq.index(node_to)
            reach_seq = reaching_times[k_init:] - reaching_times[k_init]
            leav_seq = leaving_times[k_init:] - reaching_times[k_init]
            
            offset = fleet.network.t(node_from, node_to,  leaving_times[k_init-1])*(1-eta)
            
            reach_seq += offset
            leav_seq += offset
            
           # print('reach seq', reach_seq)
            #print('leave seq', leav_seq)
            #print('node from', node_from, ' node to', node_to)
            #print('eta', eta)
            #print('offset', offset)
        #print()
        if k_init == len(node_seq) - 1:
            print('case last node')
            cp[id_ev] = (-1, 0, 0, 0)
        else:
            for k, reach_time in enumerate(reach_seq):
                k_crit = k_init + k
                remaining_customers_ahead_cp = sum([1 for node in node_seq if fleet.network.isCustomer(node)])
                if remaining_customers_ahead_cp <= consider_until:
                    cp[id_ev] = (-1, 0, 0, 0)
                    break
                elif reach_time > safe_time:
                    cp[id_ev] = (k_crit, ev.state_leaving[0, k_crit], ev.state_leaving[1, k_crit], ev.state_leaving[2, k_crit])
                    break
    return cp

### Show previous values

In [9]:
dataTree_realtime = ET.parse(filePath_realtime)

In [10]:
observation_time = .5   # in minutes
ga_time = 1.

# instantiate and show previous values
fleet = from_xml(filePath_realtime)

# Get previous routes
routes = {}
_fleet = dataTree_realtime.find('fleet')
for _vehicle in _fleet:
    ev_id = int(_vehicle.get('id'))
    _cp = _vehicle.find('critical_point')
    k, x1, x2, x3 = int(_cp.get('k')), float(_cp.get('x1')), float(_cp.get('x2')), float(_cp.get('x3'))
    if k != -1:
        prev_route = (tuple(int(x.get('Sk')) for x in _vehicle.find('previous_route')[k:]), 
                     tuple(float(x.get('Lk')) for x in _vehicle.find('previous_route')[k:]))

        routes[ev_id] = (prev_route, x1, x2, x3)

# Update vehicles
fleet.update_from_xml(filePath_realtime, do_network=True)
fleet.set_routes_of_vehicles(routes)
fleet.create_optimization_vector()

# Show observation with previous values
observation = observe(fleet, observation_time+ga_time)
print(f'  Observation\n {observation}\n')

# Show critical points with previous values
cp = critical_points(fleet, observation, observation_time+ga_time, 3)
print(f'  Old critical points\n {cp}')

  Observation
 {0: (1, 0.15120967741935545, 0, 9), 1: (1, 0.07949125596184418, 0, 6)}

  Old critical points
 {0: (1, 617.4888309982737, 73.7, 1.75), 1: (1, 753.5300605292763, 75.03, 1.22)}


### Perturbate network

In [34]:
max_tt_pert = 3
max_ec_pert = 2

_network = dataTree_realtime.find('network')
_nodes = _network.find('nodes')
_edges = _network.find('edges')

# Modify travel times and energy consumption
for _node_from in _edges:
    for _node_to in _node_from:
        tt = float(_node_to.get('travel_time'))
        ec = float(_node_to.get('energy_consumption'))
        
        tt_dev = min(.8*tt, max_tt_pert)
        tt += random.uniform(-tt_dev, tt_dev)
        
        ec_dev = min(.8*ec, max_ec_pert)
        ec += random.uniform(-ec_dev, ec_dev)

        _node_to.set('travel_time', str(tt))
        _node_to.set('energy_consumption', str(ec))
        
# Save
dataTree_realtime.write(filePath_realtime)
if print_pretty:
    xml_pretty = xml.dom.minidom.parse(filePath_realtime).toprettyxml()
    with open(filePath_realtime, 'w') as file:
        file.write(xml_pretty)

### Show new critical points

In [35]:
# Update vehicles
fleet.update_from_xml(filePath_realtime, do_network=True)
fleet.set_routes_of_vehicles(routes)
fleet.create_optimization_vector()

# Show observation with previous values
observation = observe(fleet, observation_time+ga_time)
print(f'  Observation\n {observation}\n')

# Show critical points with previous values
cp = critical_points(fleet, observation, observation_time+ga_time)
print(f'  New critical points\n {cp}')

  Observation
 {0: (1, 0.09073121890592566, 1, 3), 1: (1, 0.08307517304846906, 2, 5)}

  New critical points
 {0: (-1, 0, 0, 0), 1: (-1, 0, 0, 0)}


### Update critical points in tree and save

In [36]:
# update
for _vehicle in _fleet:
    id_ev = int(_vehicle.get('id'))
    if id_ev in fleet.vehicles_to_route:
        _cp = _vehicle.find('critical_point')
        _cp.set('k', str(cp[id_ev][0]))
        _cp.set('x1', str(cp[id_ev][1]))
        _cp.set('x2', str(cp[id_ev][2]))
        _cp.set('x3', str(cp[id_ev][3]))

# Save
dataTree_realtime.write(filePath_realtime)
if print_pretty:
    xml_pretty = xml.dom.minidom.parse(filePath_realtime).toprettyxml()
    with open(filePath_realtime, 'w') as file:
        file.write(xml_pretty)